In [36]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy

bRedownloadLatest = True # will download feature class from agol, otherwise will use last download
bExportFeatureClass = True # export feature class
bClearFlags = True # will clear all flag fields from backup
bClearOverrides = False # will clear all override fields from backup

In [37]:
dfSegmentSource = pd.DataFrame([
    ['Segments_State_20231221_Draft.shp' , [0,2,3,4,5]],
    ['WFv901_Segments_20240226_Draft.shp', [1]        ]
],columns=('SOURCE','SUBAREAID'))

dfSegmentSource = dfSegmentSource.explode('SUBAREAID')

seg_cols = ['SEGID','CO_FIPS','PLANAREA','SUBAREAID','F_AREA','SHAPE']

# get observed volume source (SHOULD ONLY HAVE ONE SOURCE PER SUBAREA)
dfObsVolSource = pd.read_csv('intermediate/obsvolsource.csv')
dfObsVolSource

,SOURCE,SUBAREAID
0,AADTHistory.xlsx,0
1,AADTHistory.xlsx,2
2,AADTHistory.xlsx,3
3,AADTHistory.xlsx,4
4,AADTHistory.xlsx,5
5,WFv901_Segments_20240226_Draft.shp,1


In [38]:
# Prep flag categories

# flag name must be short enough to be in dbf column name
dfFlags = pd.DataFrame([
    ['FL_REV'   , 'Not reviewed'                    , "([NOTES]=='') & ([NOTES_FURR]=='')"],
    ['FL_LTPRV' , 'Less than previous forecast year', "(([MF2023]+[ADJ2023])<([M2019]+[aadtAdjFac])) | (([MF2028]+[ADJ2028])<([MF2023]+[ADJ2023])) | (([MF2032]+[ADJ2032])<([MF2028]+[ADJ2028])) | (([MF2042]+[ADJ2042])<([MF2032]+[ADJ2032])) | (([MF2050]+[ADJ2050])<([MF2042]+[ADJ2042]))"],
    ['FL_ZERO'  , 'Zero volume'                     , "(([MF2023]+[ADJ2023])<=0) | (([MF2028]+[ADJ2028])<=0) | (([MF2032]+[ADJ2032])<=0) | (([MF2042]+[ADJ2042])<=0) | (([MF2050]+[ADJ2050])<=0)"],
    ['FL_HIADJ' , 'Large model adjustment factor '  , "((([aadtAdjFac]/([M2019]+[aadtAdjFac]))>0.95) | (([aadtAdjFac]/([M2019]+[aadtAdjFac]))<-0.95))"],
    ['FL_50LT19', 'HPMS: 2050 < 2019'               , "(([MF2050]+[ADJ2050])<([M2019]+[aadtAdjFac]))"],
    ['FL_503X19', 'HPMS: 2050 > 3 x 2019'           , "(([MF2050]+[ADJ2050])>(3*([M2019]+[aadtAdjFac])))"],
    ['FL_FURREV', 'Further Review'                  , "([NOTES_FURR]!='')"],
    ['FL_SEG'   , 'Segment Note'                    , "([NOTES_SEG]!='')"]
], columns=('flagName','flagDescription','flagCriteria'))

dfFlags.to_json('_site-art-of-forecasting/data/flags.json', orient='records')

In [39]:
# CO_FIPS
county_mapping = {
     1: {'CO_NAME': 'Beaver'    },
     3: {'CO_NAME': 'Box Elder' },
     5: {'CO_NAME': 'Cache'     },
     7: {'CO_NAME': 'Carbon'    },
     9: {'CO_NAME': 'Daggett'   },
    11: {'CO_NAME': 'Davis'     },
    13: {'CO_NAME': 'Duchesne'  },
    15: {'CO_NAME': 'Emery'     },
    17: {'CO_NAME': 'Garfield'  },
    19: {'CO_NAME': 'Grand'     },
    21: {'CO_NAME': 'Iron'      },
    23: {'CO_NAME': 'Juab'      },
    25: {'CO_NAME': 'Kane'      },
    27: {'CO_NAME': 'Millard'   },
    29: {'CO_NAME': 'Morgan'    },
    31: {'CO_NAME': 'Piute'     },
    33: {'CO_NAME': 'Rich'      },
    35: {'CO_NAME': 'Salt Lake' },
    37: {'CO_NAME': 'San Juan'  },
    39: {'CO_NAME': 'Sanpete'   },
    41: {'CO_NAME': 'Sevier'    },
    43: {'CO_NAME': 'Summit'    },
    45: {'CO_NAME': 'Tooele'    },
    47: {'CO_NAME': 'Uintah'    },
    49: {'CO_NAME': 'Utah'      },
    51: {'CO_NAME': 'Wasatch'   },
    53: {'CO_NAME': 'Washington'},
    55: {'CO_NAME': 'Wayne'     },
    57: {'CO_NAME': 'Weber'     }
}


# Create JSONs

In [40]:
# convert csvs to jsons
lstSegidFiles = ['model-forecasts', 'previous-forecasts']
lstNonSegidFiles = ['aadt-sources', 'previous-forecasts-sources', 'projection-groups']

for file in lstSegidFiles + lstNonSegidFiles:
    # Read CSV file
    pd.read_csv('intermediate/' + file + '.csv').to_json('_site-art-of-forecasting/data/' + file + '.json', orient='records')

In [41]:
dfSegSub =  pd.read_csv('intermediate/model-forecasts.csv',usecols=['SEGID','SUBAREAID']).drop_duplicates()
display(dfSegSub)

,SEGID,SUBAREAID
0,1831_000.0,0.0
6,1180_002.5,2.0
12,0006_000.0,0.0
13,0006_000.7,0.0
14,0006_016.0,0.0
...,...,...
47076,3218_006.4,5.0
47077,UDOT_7008,5.0
47078,UDOT_7043,5.0
47769,UDOT_7006,5.0


In [44]:
# AADT 
dfAADT = pd.read_csv('intermediate/aadt.csv')
dfAADTWithSub = pd.merge(dfAADT, dfSegSub, on="SEGID")
display(dfAADTWithSub)

# Linear Forecast
dfLF =  pd.read_csv('intermediate/linear-forecasts.csv')
dfLFWithSub = pd.merge(dfLF, dfSegSub, on="SEGID")
display(dfLFWithSub)


,SOURCE,SEGID,YEAR,AADT,SUBAREAID
0,AADTHistory.xlsx,0006_000.0,1981,325,0.0
1,AADTHistory.xlsx,0006_000.0,1982,335,0.0
2,AADTHistory.xlsx,0006_000.0,1983,430,0.0
3,AADTHistory.xlsx,0006_000.0,1984,580,0.0
4,AADTHistory.xlsx,0006_000.0,1985,585,0.0
...,...,...,...,...,...
282199,WFv901_Segments_20240226_Draft.shp,WFRC_8467,2018,26161,1.0
282200,WFv901_Segments_20240226_Draft.shp,WFRC_8467,2019,26632,1.0
282201,WFv901_Segments_20240226_Draft.shp,WFRC_8467,2020,23782,1.0
282202,WFv901_Segments_20240226_Draft.shp,WFRC_8467,2021,25780,1.0


,SEGID,SOURCE,PROJGRP,YEAR,linForecast,SUBAREAID
0,0006_000.0,AADTHistory.xlsx,Since 1981,1981,461,0.0
1,0006_000.0,AADTHistory.xlsx,Since 1981,2023,365,0.0
2,0006_000.0,AADTHistory.xlsx,Since 1981,2028,353,0.0
3,0006_000.0,AADTHistory.xlsx,Since 1981,2032,344,0.0
4,0006_000.0,AADTHistory.xlsx,Since 1981,2042,321,0.0
...,...,...,...,...,...,...
314185,WFRC_8467,WFv901_Segments_20240226_Draft.shp,Since 2001 w/o 2020,2023,27464,1.0
314186,WFRC_8467,WFv901_Segments_20240226_Draft.shp,Since 2001 w/o 2020,2028,29782,1.0
314187,WFRC_8467,WFv901_Segments_20240226_Draft.shp,Since 2001 w/o 2020,2032,31636,1.0
314188,WFRC_8467,WFv901_Segments_20240226_Draft.shp,Since 2001 w/o 2020,2042,36272,1.0


In [46]:
# filter by on used
dfAADTWithSubObsVolSource = pd.merge(dfAADTWithSub,dfObsVolSource, on=("SOURCE","SUBAREAID"))
dfAADTWithSubObsVolSource.drop(columns=['SOURCE','SUBAREAID'],inplace=True)
dfAADTWithSubObsVolSource.to_json('_site-art-of-forecasting/data/aadt.json', orient='records')
display(dfAADTWithSubObsVolSource)

# filter by on used
dfLFWithSubObsVolSource = pd.merge(dfLFWithSub,dfObsVolSource, on=("SOURCE","SUBAREAID"))
dfLFWithSubObsVolSource.drop(columns=['SOURCE','SUBAREAID'],inplace=True)
dfLFWithSubObsVolSource.to_json('_site-art-of-forecasting/data/linear-forecasts.json', orient='records')
display(dfLFWithSubObsVolSource)

,SEGID,YEAR,AADT
0,0006_000.0,1981,325
1,0006_000.0,1982,335
2,0006_000.0,1983,430
3,0006_000.0,1984,580
4,0006_000.0,1985,585
...,...,...,...
183536,3136_001.0,2018,500
183537,3136_001.0,2019,500
183538,3136_001.0,2020,500
183539,3136_001.0,2021,500


,SEGID,PROJGRP,YEAR,linForecast
0,0006_000.0,Since 1981,1981,461
1,0006_000.0,Since 1981,2023,365
2,0006_000.0,Since 1981,2028,353
3,0006_000.0,Since 1981,2032,344
4,0006_000.0,Since 1981,2042,321
...,...,...,...,...
124213,3136_001.0,Since 2001 w/o 2020,2023,504
124214,3136_001.0,Since 2001 w/o 2020,2028,508
124215,3136_001.0,Since 2001 w/o 2020,2032,511
124216,3136_001.0,Since 2001 w/o 2020,2042,518


# Create Segment Feature Class

In [9]:
dfSegmentSource

,SOURCE,SUBAREAID
0,Segments_State_20231221_Draft.shp,0
0,Segments_State_20231221_Draft.shp,2
0,Segments_State_20231221_Draft.shp,3
0,Segments_State_20231221_Draft.shp,4
0,Segments_State_20231221_Draft.shp,5
1,WFv901_Segments_20240226_Draft.shp,1


In [10]:
sdfSegmentsCombined = pd.DataFrame()

for source in dfSegmentSource['SOURCE'].drop_duplicates():
    print(source)
    _df =  pd.DataFrame.spatial.from_featureclass('data/segments/' + source)

    _subareasFilter = dfSegmentSource[dfSegmentSource['SOURCE']==source]['SUBAREAID'].drop_duplicates()

    # filter out only segments with SUBAREAID that matches source
    _df_filtered = _df[_df['SUBAREAID'].isin(_subareasFilter)]

    if 'F_AREA' not in _df_filtered.columns:
        # If 'F_AREA' doesn't exist, create it and set its value equal to 'PLAN_AREA'
        _df_filtered['F_AREA'] = _df_filtered['PLANAREA']

    _df_filtered = _df_filtered[seg_cols]

    sdfSegmentsCombined = pd.concat([_df_filtered,sdfSegmentsCombined])

sdfSegmentsCombined


Segments_State_20231221_Draft.shp
WFv901_Segments_20240226_Draft.shp


,SEGID,CO_FIPS,PLANAREA,SUBAREAID,F_AREA,SHAPE
0,0006_141.0,49.0,MAG,1.0,MAG,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,0006_146.9,49.0,MAG,1.0,MAG,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,0006_149.9,49.0,MAG,1.0,MAG,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,0006_150.6,49.0,MAG,1.0,MAG,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,0006_152.6,49.0,MAG,1.0,MAG,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...
8427,UDOT_7090,45.0,UDOT,0.0,UDOT,"{""paths"": [[[388880.12200000044, 4497794.5009]..."
8428,UDOT_7091,45.0,UDOT,0.0,UDOT,"{""paths"": [[[388884.62009999994, 4491344.6401]..."
8429,UDOT_7092,45.0,UDOT,0.0,UDOT,"{""paths"": [[[392102.12590000033, 4494203.1196]..."
8430,UDOT_7093,45.0,UDOT,0.0,UDOT,"{""paths"": [[[394365.53309999965, 4504536.84039..."


In [11]:
duplicateSegsInTdms = pd.read_csv('intermediate/duplicateSegsInTdms.csv')
duplicateSegsInTdms

,SEGID,removeSUBAREAID
0,0015_042.2,3
1,0065_002.4,0
2,0150_029.4,0
3,0150_030.0,0
4,0150_033.3,0
5,1822_000.0,0
6,2863_000.0,4
7,2865_019.4,4


In [12]:
# Merge the dataframes on 'SEGID' and 'SUBAREAID'/'removeSUBAREAID', indicating records to remove
merged_df = pd.merge(sdfSegmentsCombined, duplicateSegsInTdms, left_on=['SEGID', 'SUBAREAID'], right_on=['SEGID', 'removeSUBAREAID'], how='left', indicator=True)

# Filter out the records that have a match
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the columns added from df1 and the merge indicator to get back to the original dfMdlVolWithFac format
final_df = filtered_df.drop(columns=['removeSUBAREAID', '_merge'])

# final_df will be your dfMdlVolWithFac dataframe with the matched records removed
sdfSegments = final_df
sdfSegments

,SEGID,CO_FIPS,PLANAREA,SUBAREAID,F_AREA,SHAPE
0,0006_141.0,49.0,MAG,1.0,MAG,"{'paths': [[[405824.11000000034, 4423860.33], ..."
1,0006_146.9,49.0,MAG,1.0,MAG,"{'paths': [[[413442.55030000024, 4422753.7282]..."
2,0006_149.9,49.0,MAG,1.0,MAG,"{'paths': [[[418330.7999999998, 4422866], [418..."
3,0006_150.6,49.0,MAG,1.0,MAG,"{'paths': [[[419421.8803000003, 4422872.2963],..."
4,0006_152.6,49.0,MAG,1.0,MAG,"{'paths': [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...
9282,UDOT_7090,45.0,UDOT,0.0,UDOT,"{'paths': [[[388880.12200000044, 4497794.5009]..."
9283,UDOT_7091,45.0,UDOT,0.0,UDOT,"{'paths': [[[388884.62009999994, 4491344.6401]..."
9284,UDOT_7092,45.0,UDOT,0.0,UDOT,"{'paths': [[[392102.12590000033, 4494203.1196]..."
9285,UDOT_7093,45.0,UDOT,0.0,UDOT,"{'paths': [[[394365.53309999965, 4504536.84039..."


In [13]:
dfDuplicates = sdfSegments[sdfSegments.duplicated(subset='SEGID', keep=False)]
dfDuplicates

,SEGID,CO_FIPS,PLANAREA,SUBAREAID,F_AREA,SHAPE


# Get Latest Download with Adjustments to be applied to Segments

In [14]:
if bRedownloadLatest:

    # Define the URL
    url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0/query"

    # Base parameters for the request
    params = {
        "f": "json",
        "where": "1=1",
        "outFields": "*",
        "returnGeometry": "false",  # Assuming you only want attributes
    }

    all_features = []

    # Pagination parameters
    batch_size = 1000
    offset = 0

    while True:
        # Adjust parameters for pagination
        params['resultOffset'] = offset
        params['resultRecordCount'] = batch_size

        # Send the request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])
            
            # Check if no features were returned, which means we're done
            if not features:
                break
            
            # Append features to the master list
            all_features.extend(features)
            
            # Increase the offset for next iteration
            offset += batch_size

        else:
            print("Error fetching data:", response.status_code)
            break

    # Extract attribute table
    attribute_table = [feature['attributes'] for feature in all_features]

    # Convert to DataFrame
    df = pd.DataFrame(attribute_table)

    from datetime import datetime

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"backup\\forecastsegments_backup_{timestamp}.csv"

    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
else:
    print("didn't download... will use latest")

Data saved to backup\forecastsegments_backup_20240306_212108.csv


In [15]:
# Define the URL
url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/forecasts_logfile/FeatureServer/0/query"

# Base parameters for the request
params = {
    "f": "json",
    "where": "1=1",
    "outFields": "*",
    "returnGeometry": "false",  # If it's a stand-alone table, geometry isn't required.
    "outSR": "4326",  # Specify output spatial reference if needed
    "returnExceededLimitFeatures": "true"
}

all_features = []

# Pagination parameters
batch_size = 2000
offset = 0

while True:
    # Adjust parameters for pagination
    params['resultOffset'] = offset
    params['resultRecordCount'] = batch_size

    # Send the request
    response = requests.post(url, params=params)  # Use POST since some servers may have GET length limits

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        
        # Check if no features were returned, which means we're done
        if not features:
            break
        
        # Append features to the master list
        all_features.extend(features)
        
        # Increase the offset for next iteration
        offset += batch_size

    else:
        print("Error fetching data:", response.status_code)
        break

# Extract attribute table
attribute_table = [feature['attributes'] for feature in all_features]

# Convert to DataFrame
df = pd.DataFrame(attribute_table)

from datetime import datetime

# Generate a filename with the current timestamp in the specified folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"backup\\logfile_backup_{timestamp}.csv"

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)
print(f"Data saved to {filename}")

Data saved to backup\logfile_backup_20240306_212115.csv


In [16]:
#import datetime
#
## empty log file for AGOL
#dfLogFile = pd.DataFrame([
#    ['dummy','dummy',0,0,0,0,0,'dummy',datetime.datetime.now()]
#],columns=('SEGID','EDITKEY','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','NOTES','TIMESTAMP'))
#
#dfLogFile.to_csv('results/forecasts-logfile.csv')
#
#dfLogFile

In [17]:
# Get data from last backup

import os

# List all backup files in the directory
backup_dir = "backup\\"
all_files = [f for f in os.listdir(backup_dir) if f.startswith('forecastsegments_backup_') and f.endswith('.csv')]

# Sort the files based on their timestamps
sorted_files = sorted(all_files, reverse=True)  # Latest timestamp will be first

# Check if there are any backup files
if sorted_files:
    latest_file = os.path.join(backup_dir, sorted_files[0])
    # Load the most recent backup into a dataframe
    dfLatest = pd.read_csv(latest_file,low_memory=False)

    # when duplicate errors we need to get rid of extra data
    dfLatest = dfLatest.groupby('SEGID').first().reset_index()

    print(f"Loaded {latest_file} into a DataFrame")

    if bClearFlags:
        # Specify columns to delete
        columns_to_delete = [col for col in dfLatest.columns if col.startswith('FL_')]

        # Remove the specified columns from df_filtered
        dfLatest.drop(columns_to_delete, axis=1, inplace=True)
    
        print(f"Dropped FL_ columns")

    if bClearOverrides:
        # Specify columns to delete
        columns_to_delete = [col for col in dfLatest.columns if col.startswith('OV_')]

        # Remove the specified columns from df_filtered
        dfLatest.drop(columns_to_delete, axis=1, inplace=True)
    
        print(f"Dropped OV_ columns")
else:
    dfLatest = pd.DataFrame()
    print("No backup files found in the directory.")

# You can then use dfLatest as your DataFrame


Loaded backup\forecastsegments_backup_20240306_212108.csv into a DataFrame
Dropped FL_ columns


In [18]:
display(sdfSegments)

,SEGID,CO_FIPS,PLANAREA,SUBAREAID,F_AREA,SHAPE
0,0006_141.0,49.0,MAG,1.0,MAG,"{'paths': [[[405824.11000000034, 4423860.33], ..."
1,0006_146.9,49.0,MAG,1.0,MAG,"{'paths': [[[413442.55030000024, 4422753.7282]..."
2,0006_149.9,49.0,MAG,1.0,MAG,"{'paths': [[[418330.7999999998, 4422866], [418..."
3,0006_150.6,49.0,MAG,1.0,MAG,"{'paths': [[[419421.8803000003, 4422872.2963],..."
4,0006_152.6,49.0,MAG,1.0,MAG,"{'paths': [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...
9282,UDOT_7090,45.0,UDOT,0.0,UDOT,"{'paths': [[[388880.12200000044, 4497794.5009]..."
9283,UDOT_7091,45.0,UDOT,0.0,UDOT,"{'paths': [[[388884.62009999994, 4491344.6401]..."
9284,UDOT_7092,45.0,UDOT,0.0,UDOT,"{'paths': [[[392102.12590000033, 4494203.1196]..."
9285,UDOT_7093,45.0,UDOT,0.0,UDOT,"{'paths': [[[394365.53309999965, 4504536.84039..."


In [19]:
dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')

dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'DYVOL' + dfModVolAdj['YEAR'].str.replace(".0","", regex=False)

dfModVolAdj_pvDYVOL = dfModVolAdj.pivot_table(index=['SEGID','SUBAREAID','FAC_WDAVG','FAC_SPR','FAC_FAL','FAC_SPRFAL','aadtAdjFac'], columns='YEAR', values='DY_VOL')
dfModVolAdj_pvDYVOL.reset_index(inplace=True)
dfModVolAdj_pvDYVOL

display(dfModVolAdj_pvDYVOL)


YEAR,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,DYVOL2032,DYVOL2042,DYVOL2050
0,0006_000.0,0.0,0.9840,1.0276,1.0316,1.02960,286,126.5,99.7,101.7,103.6,107.9,111.7
1,0006_000.7,0.0,0.9840,1.0276,1.0316,1.02960,198,213.6,216.5,215.2,215.4,216.2,216.6
2,0006_016.0,0.0,0.9840,1.0276,1.0316,1.02960,337,76.9,73.9,69.9,67.9,65.8,62.5
3,0006_046.0,0.0,0.9840,1.0276,1.0316,1.02960,294,76.9,73.9,69.9,67.9,65.8,62.5
4,0006_060.2,0.0,0.9840,1.0276,1.0316,1.02960,290,80.3,77.4,73.2,71.2,69.6,66.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8404,WFRC_8465,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,NaN,NaN,44066.0,46952.8,46751.7
8405,WFRC_8466,1.0,1.0946,1.0117,1.0095,1.01060,0,9082.2,9055.3,10036.0,11066.0,13533.5,15608.7
8406,WFRC_8467,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,17522.8,21032.6,22212.8,25093.4,26571.9
8407,WFRC_8471,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,NaN,NaN,15006.8,18175.7,20343.9


In [20]:
# read in intermediate data
dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'M' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadt = dfModVolAdj.pivot_table(index=['SEGID','SUBAREAID','FAC_WDAVG','FAC_SPR','FAC_FAL','FAC_SPRFAL','aadtAdjFac'], columns='YEAR', values='modAadt')
dfModVolAdj_pvModAadt.reset_index(inplace=True)

dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')

dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'MF' + dfModVolAdj['YEAR'].str.replace(".0","", regex=False)

dfModVolAdj_pvModAadtAdj = dfModVolAdj.pivot_table(index=['SEGID','SUBAREAID','FAC_WDAVG','FAC_SPR','FAC_FAL','FAC_SPRFAL','aadtAdjFac'], columns='YEAR', values='modForecast')
dfModVolAdj_pvModAadtAdj.reset_index(inplace=True)

dfModVolAdj_pv1 = pd.DataFrame.merge(dfModVolAdj_pvModAadt, dfModVolAdj_pvModAadtAdj, on=('SEGID','SUBAREAID','FAC_WDAVG','FAC_SPR','FAC_FAL','FAC_SPRFAL','aadtAdjFac'))


dfModVolAdj = pd.DataFrame.merge(dfModVolAdj_pvDYVOL, dfModVolAdj_pv1, on=('SEGID','SUBAREAID','FAC_WDAVG','FAC_SPR','FAC_FAL','FAC_SPRFAL','aadtAdjFac'))
display(dfModVolAdj)

#dfModVolAdj.columns = [str(col).split('.')[0] if '.' in str(col) else str(col) for col in dfModVolAdj.columns]


YEAR,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,M2028,M2032,M2042,M2050,MF2019,MF2023,MF2028,MF2032,MF2042,MF2050
0,0006_000.0,0.0,0.9840,1.0276,1.0316,1.02960,286,126.5,99.7,101.7,...,103.0,105.0,110.0,114.0,400.0,400.0,400.0,400.0,400.0,400.0
1,0006_000.7,0.0,0.9840,1.0276,1.0316,1.02960,198,213.6,216.5,215.2,...,219.0,219.0,220.0,220.0,400.0,400.0,400.0,400.0,400.0,400.0
2,0006_016.0,0.0,0.9840,1.0276,1.0316,1.02960,337,76.9,73.9,69.9,...,71.0,69.0,67.0,64.0,400.0,400.0,400.0,400.0,400.0,400.0
3,0006_046.0,0.0,0.9840,1.0276,1.0316,1.02960,294,76.9,73.9,69.9,...,71.0,69.0,67.0,64.0,350.0,350.0,350.0,350.0,350.0,350.0
4,0006_060.2,0.0,0.9840,1.0276,1.0316,1.02960,290,80.3,77.4,73.2,...,74.0,72.0,71.0,67.0,350.0,350.0,350.0,350.0,350.0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8404,WFRC_8465,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,NaN,NaN,...,NaN,40339.0,42981.0,42797.0,NaN,NaN,NaN,40500.0,43000.0,43000.0
8405,WFRC_8466,1.0,1.0946,1.0117,1.0095,1.01060,0,9082.2,9055.3,10036.0,...,9169.0,10110.0,12364.0,14260.0,8300.0,8300.0,9200.0,10000.0,12500.0,14500.0
8406,WFRC_8467,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,17522.8,21032.6,...,19254.0,20334.0,22971.0,24324.0,NaN,16000.0,19500.0,20500.0,23000.0,24500.0
8407,WFRC_8471,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,NaN,NaN,...,NaN,13737.0,16638.0,18623.0,NaN,NaN,NaN,13500.0,16500.0,18500.0


In [21]:
dfModVolAdj[dfModVolAdj['SUBAREAID']==1]

YEAR,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,M2028,M2032,M2042,M2050,MF2019,MF2023,MF2028,MF2032,MF2042,MF2050
22,0006_141.0,1.0,0.9840,1.0276,1.0316,1.02960,-32,1524.0,1655.0,1737.0,...,1765.0,1881.0,2128.0,2399.0,1500.0,1600.0,1700.0,1800.0,2100.0,2400.0
23,0006_146.9,1.0,0.9840,1.0276,1.0316,1.02960,-36,1528.0,1661.0,1747.0,...,1775.0,1888.0,2132.0,2403.0,1500.0,1700.0,1700.0,1900.0,2100.0,2400.0
24,0006_149.9,1.0,0.9840,1.0276,1.0316,1.02960,857,1558.4,1680.1,2049.6,...,2083.0,2090.0,2652.0,5259.0,2400.0,2600.0,2900.0,2900.0,3500.0,6100.0
25,0006_150.6,1.0,0.9840,1.0276,1.0316,1.02960,664,1748.2,1867.2,2241.2,...,2278.0,2291.0,2894.0,5486.0,2400.0,2600.0,2900.0,3000.0,3600.0,6200.0
26,0006_152.6,1.0,0.9840,1.0276,1.0316,1.02960,-160,2535.8,2634.8,3002.5,...,3051.0,3093.0,3709.0,6098.0,2400.0,2500.0,2900.0,2900.0,3500.0,5900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8404,WFRC_8465,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,NaN,NaN,...,NaN,40339.0,42981.0,42797.0,NaN,NaN,NaN,40500.0,43000.0,43000.0
8405,WFRC_8466,1.0,1.0946,1.0117,1.0095,1.01060,0,9082.2,9055.3,10036.0,...,9169.0,10110.0,12364.0,14260.0,8300.0,8300.0,9200.0,10000.0,12500.0,14500.0
8406,WFRC_8467,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,17522.8,21032.6,...,19254.0,20334.0,22971.0,24324.0,NaN,16000.0,19500.0,20500.0,23000.0,24500.0
8407,WFRC_8471,1.0,1.0924,1.0104,1.0243,1.01735,0,NaN,NaN,NaN,...,NaN,13737.0,16638.0,18623.0,NaN,NaN,NaN,13500.0,16500.0,18500.0


In [22]:
dfModVolAdj[dfModVolAdj['SEGID']=='MAG_6501']

YEAR,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,M2028,M2032,M2042,M2050,MF2019,MF2023,MF2028,MF2032,MF2042,MF2050
8065,MAG_6501,1.0,0.984,1.0276,1.0316,1.0296,0,0.0,0.0,243.7,...,248.0,260.0,289.0,319.0,0.0,0.0,250.0,250.0,300.0,300.0


In [23]:
# join segment data and forecast data
_sdf = pd.DataFrame.merge(dfModVolAdj, sdfSegments, on=('SEGID','SUBAREAID'), how='right')

# add columns for Adjustments
_sdf['MF2019'].fillna(0,inplace=True)
_sdf['MF2023'].fillna(0,inplace=True)
_sdf['MF2028'].fillna(0,inplace=True)
_sdf['MF2032'].fillna(0,inplace=True)
_sdf['MF2042'].fillna(0,inplace=True)
_sdf['MF2050'].fillna(0,inplace=True)

_sdf['ADJ2019'   ] = 0
_sdf['ADJ2023'   ] = 0
_sdf['ADJ2028'   ] = 0
_sdf['ADJ2032'   ] = 0
_sdf['ADJ2042'   ] = 0
_sdf['ADJ2050'   ] = 0
_sdf['ADJHIST'   ] = 0
_sdf['NOTES'     ] = ""
_sdf['NOTES_FURR'] = ""
_sdf['NOTES_SEG' ] = ""


# replace with latest file
if dfLatest.empty:
    print("The latest dataframe is empty.")
else:
    print("The latest dataframe is not empty.")
        
    # Specify columns to include from latest download (FLAGS WILL NOT BE INCLUDED WHEN FILTERED PRIOR TO THIS STEP)
    columns_to_include = [col for col in dfLatest.columns if col.startswith('ADJ') or col.startswith('FL_') or col.startswith('OV_') or col == 'SEGID' or col == 'NOTES' or col == 'NOTES_FURR' or col == 'NOTES_SEG']

    # Filter df_filtered to only have the specified columns
    dfLatest_filtered = dfLatest[columns_to_include]

    # Merge sdfData with df_filtered on SEGID
    _sdfWithLatest = _sdf.merge(dfLatest_filtered, on='SEGID', how='left', suffixes=('_delete', ''))

    # Drop any duplicate columns (those with '_delete' suffix) after the merge 
    for column in _sdfWithLatest.columns:
        if column.endswith('_delete'):
            _sdfWithLatest.drop(column, axis=1, inplace=True)

    _sdfWithLatest['NOTES'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES'] = _sdfWithLatest['NOTES'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES']=='0'), 'NOTES'] = ''
    _sdfWithLatest['NOTES'] = _sdfWithLatest['NOTES'].str.strip()

    _sdfWithLatest['NOTES_FURR'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES_FURR'] = _sdfWithLatest['NOTES_FURR'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES_FURR']=='0'), 'NOTES_FURR'] = ''
    _sdfWithLatest['NOTES_FURR'] = _sdfWithLatest['NOTES_FURR'].str.strip()

    _sdfWithLatest['NOTES_SEG'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES_SEG'] = _sdfWithLatest['NOTES_SEG'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES_SEG']=='0'), 'NOTES_SEG'] = ''
    _sdfWithLatest['NOTES_SEG'] = _sdfWithLatest['NOTES_SEG'].str.strip()

# add flag columns
# Loop through the rows of 'dfFlags'
for index, row in dfFlags.iterrows():
    # Get the flag name from the 'flagName' column
    flag_name = row['flagName']

    # Get the flag criteria from the 'flagCriteria' column (if needed)
    flag_criteria = row['flagCriteria']

    # Add a new column to 'dfSegs' with the flag name and set it equal to evaluated criteria, prepare criteria string with replace
    expression = "_sdfWithLatest['" + flag_name + "'] = " + flag_criteria.replace("[", "_sdfWithLatest['").replace("]", "']")
    print(expression)

    # execute expression!
    exec(expression)
    #print(result)  # Output will be 11
    
    if bClearOverrides:
        display('flags overwritten')
        # add override column
        _sdfWithLatest[flag_name.replace('FL_','OV_')] = 0

_sdfWithLatest.fillna(0,inplace=True)

sdfSegmentsWData = _sdfWithLatest
sdfSegmentsWData

The latest dataframe is not empty.
_sdfWithLatest['FL_REV'] = (_sdfWithLatest['NOTES']=='') & (_sdfWithLatest['NOTES_FURR']=='')
_sdfWithLatest['FL_LTPRV'] = ((_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])<(_sdfWithLatest['M2019']+_sdfWithLatest['aadtAdjFac'])) | ((_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])<(_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])) | ((_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])<(_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])) | ((_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042'])<(_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])) | ((_sdfWithLatest['MF2050']+_sdfWithLatest['ADJ2050'])<(_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042']))
_sdfWithLatest['FL_ZERO'] = ((_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])<=0) | ((_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])<=0) | ((_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])<=0) | ((_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042'])<=0) | ((_sdfWithLatest['MF2050

,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,OV_SEG,NOTES_SEG,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG
0,0006_141.0,1.0,0.984,1.0276,1.0316,1.0296,-32.0,1524.0,1655.0,1737.0,...,0,,False,False,False,False,False,False,False,False
1,0006_146.9,1.0,0.984,1.0276,1.0316,1.0296,-36.0,1528.0,1661.0,1747.0,...,0,,False,False,False,False,False,False,False,False
2,0006_149.9,1.0,0.984,1.0276,1.0316,1.0296,857.0,1558.4,1680.1,2049.6,...,0,,False,False,False,False,False,False,False,False
3,0006_150.6,1.0,0.984,1.0276,1.0316,1.0296,664.0,1748.2,1867.2,2241.2,...,0,,False,False,False,False,False,False,False,False
4,0006_152.6,1.0,0.984,1.0276,1.0316,1.0296,-160.0,2535.8,2634.8,3002.5,...,0,,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9278,UDOT_7090,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,0,,True,False,True,False,False,False,False,False
9279,UDOT_7091,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,0,,True,False,True,False,False,False,False,False
9280,UDOT_7092,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,0,,True,False,True,False,False,False,False,False
9281,UDOT_7093,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,0,,True,False,True,False,False,False,False,False


In [24]:
sdfSegmentsWData[['SEGID','SUBAREAID','NOTES','NOTES_FURR','NOTES_SEG','FL_FURREV','OV_FURREV']]

,SEGID,SUBAREAID,NOTES,NOTES_FURR,NOTES_SEG,FL_FURREV,OV_FURREV
0,0006_141.0,1.0,smoothing,,,False,0
1,0006_146.9,1.0,smoothing,,,False,0
2,0006_149.9,1.0,large increase in 2050 due to I-15 congestion,,,False,0
3,0006_150.6,1.0,large increase in 2050 due to I-15 congestion,,,False,0
4,0006_152.6,1.0,large increase in 2050 due to I-15 congestion,,,False,0
...,...,...,...,...,...,...,...
9278,UDOT_7090,0.0,,,,False,0
9279,UDOT_7091,0.0,,,,False,0
9280,UDOT_7092,0.0,,,,False,0
9281,UDOT_7093,0.0,,,,False,0


In [25]:
sdfSegmentsWData[sdfSegmentsWData['NOTES_FURR']!=""]['NOTES_FURR']

508                double check if it should drop in 2023
1145    Adjusting to actual model counts.  Needs 19, 2...
1146                                                  Hmm
3035             Double check SE data for of nearby tazes
3153                            double check drop in 2023
                              ...                        
8196    Increased to around 1,000 to reflect the devel...
8384    Does it make sense that this segment is higher...
8385    Does it make sense that this segment is higher...
9210                                  This doesn't exist.
9250    It looks like the model may be overestimating ...
Name: NOTES_FURR, Length: 469, dtype: object

In [26]:
# Add CO_NAME and ForecastArea columns to the DataFrame using multi-index mapping
sdfSegmentsWData['CO_NAME'] = sdfSegmentsWData.apply(lambda row: county_mapping.get(row['CO_FIPS'], {}).get('CO_NAME'), axis=1)
sdfSegmentsWData

,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,NOTES_SEG,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME
0,0006_141.0,1.0,0.984,1.0276,1.0316,1.0296,-32.0,1524.0,1655.0,1737.0,...,,False,False,False,False,False,False,False,False,Utah
1,0006_146.9,1.0,0.984,1.0276,1.0316,1.0296,-36.0,1528.0,1661.0,1747.0,...,,False,False,False,False,False,False,False,False,Utah
2,0006_149.9,1.0,0.984,1.0276,1.0316,1.0296,857.0,1558.4,1680.1,2049.6,...,,False,False,False,False,False,False,False,False,Utah
3,0006_150.6,1.0,0.984,1.0276,1.0316,1.0296,664.0,1748.2,1867.2,2241.2,...,,False,False,False,False,False,False,False,False,Utah
4,0006_152.6,1.0,0.984,1.0276,1.0316,1.0296,-160.0,2535.8,2634.8,3002.5,...,,False,False,False,False,False,False,False,False,Utah
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9278,UDOT_7090,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,,True,False,True,False,False,False,False,False,Tooele
9279,UDOT_7091,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,,True,False,True,False,False,False,False,False,Tooele
9280,UDOT_7092,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,,True,False,True,False,False,False,False,False,Tooele
9281,UDOT_7093,0.0,0.000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,...,,True,False,True,False,False,False,False,False,Tooele


In [27]:
sdfSegmentsWData[sdfSegmentsWData['SEGID']=='0056_060.6']

,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,NOTES_SEG,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME
5712,0056_060.6,5.0,1.0924,1.0104,1.0243,1.01735,-1922.0,20577.0,31000.5,31388.9,...,,True,False,False,False,False,False,False,False,Iron


In [28]:
# export
#if bShapefileExport: #sdfSegmentsWData.spatial.to_featureclass('results/ForecastSegments/ForecastSegments.shp',sanitize_columns=False)
from arcgis.features import GeoAccessor

if bExportFeatureClass:
    # Define the geodatabase path
    gdb_path        = r"results\Forecasts.gdb"
    gdb_path_backup = r"backup\Forecasts_backup.gdb"

    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path):
        arcpy.CreateFileGDB_management(r"results", "Forecasts.gdb")
        
    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path_backup):
        arcpy.CreateFileGDB_management(r"backup", "Forecasts_backup.gdb")

    # Export SDF to the geodatabase as a feature class
    feature_class_name = "ForecastSegments"

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    feature_class_name_with_timestamp = f"ForecastSegments_{timestamp}"

    # file to replace
    sdfSegmentsWData.spatial.to_featureclass(location=f"{gdb_path_backup}\\{feature_class_name_with_timestamp}", sanitize_columns=False)
    print(f"Feature class {feature_class_name_with_timestamp} created in {gdb_path_backup}")

    # backup file
    sdfSegmentsWData.spatial.to_featureclass(location=f"{gdb_path}\\{feature_class_name}", sanitize_columns=False)

    print(f"Feature class {feature_class_name} created in {gdb_path}")

else:
    print ('not exported')


Feature class ForecastSegments_20240306_212117 created in backup\Forecasts_backup.gdb
Feature class ForecastSegments created in results\Forecasts.gdb


In [29]:
sdfSegmentsWData[sdfSegmentsWData['SEGID']=='1438_000.9']

,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFac,DYVOL2019,DYVOL2023,DYVOL2028,...,NOTES_SEG,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME
1550,1438_000.9,1.0,1.0924,1.0104,1.0243,1.01735,3431.0,5546.1,5767.5,6566.3,...,,False,True,False,False,False,False,False,False,Davis


In [30]:
duplicated_segid = sdfSegmentsWData[sdfSegmentsWData.duplicated('SEGID', keep=False)]['SEGID'].unique()
duplicated_segid

array([], dtype=object)

In [31]:
sdfSegmentsWData[sdfSegmentsWData['F_AREA'].isna()][['SEGID','CO_FIPS','SUBAREAID','F_AREA']]


,SEGID,CO_FIPS,SUBAREAID,F_AREA


In [32]:
# export segment JSON
jsonSegment = sdfSegmentsWData[['SEGID','PLANAREA','SUBAREAID','CO_NAME','F_AREA']]

jsonSegment.to_json('_site-art-of-forecasting/data/segments.json', orient='records')

In [33]:
jsonSegment.groupby(['F_AREA','CO_NAME','SUBAREAID'],as_index=False).agg(numSegs=('SEGID','count'))

,F_AREA,CO_NAME,SUBAREAID,numSegs
0,Cache,Cache,2.0,489
1,Dixie,Washington,3.0,615
2,Iron,Iron,5.0,249
3,MAG,Juab,1.0,1
4,MAG,Utah,1.0,1736
5,MAG,Wasatch,4.0,173
6,Summit,Summit,4.0,205
7,UDOT,Beaver,0.0,96
8,UDOT,Box Elder,0.0,196
9,UDOT,Cache,0.0,2


In [34]:
# export shapefile as well to share with UDOT

from datetime import datetime

# Generate a filename with the current timestamp in the specified folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"results/ForecastSegments_{timestamp}.shp"

sdfSegmentsWData.spatial.to_featureclass(location=filename, sanitize_columns=False)


'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\ForecastSegments_20240306_212128.shp'

In [35]:
dfLatest_filtered[~dfLatest_filtered['NOTES_SEG'].isna()]

,SEGID,ADJHIST,ADJ2019,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,OV_REV,OV_LTPRV,OV_ZERO,OV_HIADJ,OV_50LT19,OV_503X19,OV_FURREV,NOTES,NOTES_FURR,OV_SEG,NOTES_SEG
0,0006_000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Looks good.,,0,
1,0006_000.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,Looks good.,0,
2,0006_016.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,Looks good.,0,
3,0006_046.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,Looks good.,0,
4,0006_060.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,Looks good.,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,WFRC_8469,0,0,0,0,0,0,0,0,0,1,0,0,0,0,none,,0,
9282,WFRC_8470,0,0,0,0,0,0,0,0,0,1,0,0,0,0,none,,0,
9283,WFRC_8471,0,0,0,0,0,0,0,0,0,1,0,0,1,0,new connectivity in 2032,,0,
9284,WFRC_8472,0,0,0,0,0,0,0,0,0,1,0,0,0,0,none,,0,
